<a href="https://colab.research.google.com/github/antonercool/deep-learning-exam-project/blob/master/FallDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning course project - Fall Detection




###Group members


*   Anton Sihm, studentId.: 201504954
*   Peter Marcus Hoveling, studentId.: 201508876







###Purpose

This project aims to detect human actions from live video feed, in order to achieve safety, such as
alarms when elderly people fall in their apartments.








---



### Data sets

To evaluate the fall detection, dataset from realistic video surveillance settings will be used. Framerate
and resolution may vary as finding the same formatted video data is challenging


*   mViA - Fall detection dataset



## Task 0: Mounting Google Drive data


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Creating a variable to the specific files mouted

In [4]:
from pathlib import Path
root = '/content/gdrive/My Drive/' # Don't change this
data_dirname = 'Data_complete' # Change as you like
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) # should "/content/gdrive/My Drive/Data"

Validating the data directory

In [5]:
print(p)

/content/gdrive/My Drive/Data_complete


## Task 1: Preprocessing

## Utils

In [6]:
# Importing all necessary libraries
import cv2
import os
import numpy 
import random

class VideoUtils:

  def __init__(self):
    pass

  def convert_single_video_to_images(self, video_path, annotation_path):
    # Read the video from specified path
    video = cv2.VideoCapture(video_path)
    meta_data = open(annotation_path)

    start_fall_frame = meta_data.readline() 
    end_fall_frame = meta_data.readline() 

    not_falling = False
    # check if has labels
    if "," in start_fall_frame:
      print(f"does not contain : {annotation_path}")
      ## skip current
      not_falling = True
      start_fall_frame = 0
      end_fall_frame = 0
    else:
      start_fall_frame = int(start_fall_frame)
      end_fall_frame = int(end_fall_frame)  

    traning_samples = []
    training_labels = []

    currentframe = 0

    while(True):

      # reading from frame
      ret,frame = video.read()
      if ret:
        traning_samples.append(frame)

        if not_falling == False:
          # if falling - label 1
          if currentframe >= start_fall_frame and currentframe <= end_fall_frame:
            training_labels.append(1)
          # not falling - label 0
          else:
            training_labels.append(0)
        else:
          training_labels.append(0)

        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
      else:
        break

    # Release all space and windows once done
    video.release()
    cv2.destroyAllWindows()

    traning_samples = numpy.array(traning_samples)
    training_labels =  numpy.array(training_labels)

    return (traning_samples, training_labels, True)


  def _get_movie_IDs(self, root_path):
    list_video_paths = []
    annotation_path = r"/Annotation_files"
    video_path = r"/Videos"
    
    data_sub_dir = os.listdir(root_path)

    # interate each sub dir
    for dir in data_sub_dir:
      if ".ipynb" in dir:
        continue
      # for each file in sub dir
      for file in os.listdir(str(root_path) + "/" + dir + video_path):
        absolute_video_file_path = str(str(root_path) + "/" + dir + video_path + "/"+ file)
        list_video_paths.append(absolute_video_file_path)

    return list_video_paths


  # create_partition_list(p)
  def create_partition_list(self, root_path, traning_percentage=80, validation_percentage=20):
    partition = {}

    absolute_video_path_list = video_utils._get_movie_IDs(root_path)
    random.shuffle(absolute_video_path_list)

    to_index = int((len(absolute_video_path_list)+1)*(traning_percentage/100))
    from_index = int(len(absolute_video_path_list)*(traning_percentage/100)+1)

    print(f"traning {0}:{to_index}")
    print(f"validation {from_index}:{len(absolute_video_path_list)}")

    training = absolute_video_path_list[0:to_index]
    validation = absolute_video_path_list[from_index:]

    total_frames_training = 0
    total_frames_validation = 0
    
    for image_path in training:
      cap= cv2.VideoCapture(image_path)
      total_frames_training += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for image_path in validation:
      cap= cv2.VideoCapture(image_path)
      total_frames_validation += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    
    partition['Training'] = (training, total_frames_training)
    partition['Validation'] = (validation, total_frames_validation)

    return partition




    






### Testing Video ultils


In [ ]:
video_utils = VideoUtils()
traning_samples, training_labels, is_ok = video_utils.convert_single_video_to_images(str(p) + "/Office/Videos/video (6).avi", str(p) + "/Office/Annotation_files/video (6).txt")
print(traning_samples.shape)

absolute_video_path_list = video_utils._get_movie_IDs(p)

partition = video_utils.create_partition_list(p)

print(partition['Training'][1])
print(partition['Validation'][1])


(415, 240, 320, 3)
traning 0:152
validation 153:190


### Dataloader

In [ ]:
import numpy as np
import keras
from tensorflow.keras.utils import Sequence

class FallingDataGenerator(Sequence):

  def __init__(self, movie_path_list, total_movie_frames, batch_size=32, dim=(240, 320), n_channels=3, n_classes=2, shuffle=True): 
      'Initialization'
      self.dim = dim
      self.batch_size = batch_size
      self.n_channels = n_channels
      self.n_classes = n_classes
      self.shuffle = shuffle
      self.movie_path_list = movie_path_list
      self.total_movie_frames = total_movie_frames
      self.current_video_data = 0# ((415, 240, 320, 3)(415)) # 0-32, 32-64 ...... 400-415    
      self.current_video_frame = 0

      # init by shuffle
      self.on_epoch_end()

  # Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike
  ## TODO shuffle path list
  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.movie_path_list))
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

  def get_indexes(self):
    return self.indexes

  def __data_generation(self, list_IDs_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    ## //TODO Can use multiple cores here
    # Generate data
    #for i, ID in enumerate(list_IDs_temp):
    #    # Store sample
    #    X[i,] = np.load('data/' + ID + '.npy')
#
    #    # Store class
    #    y[i] = self.labels[ID]

    return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

  def __len__(self):
    'Denotes the number of batches per epoch'
    ## Total number of frames in all videos / batchsize
    return int(np.floor(len(self.total_movie_frames) / self.batch_size))


  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    #list_IDs_temp = [self.movie_path_list[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(movie_path_list)

    return X, y


### Testing data loader


In [ ]:
# Parameters
params = {'batch_size': 32,
          'dim': (240, 320),
          'n_channels': 3,
          'n_classes': 2, 
          'shuffle': True}

#utils_video
video_utils = VideoUtils()

# Datasets
partition = video_utils.create_partition_list(p)


# Generators
training_generator = FallingDataGenerator(partition['Training'][0], partition['Training'][1], **params)
validation_generator = FallingDataGenerator(partition['Validation'][0], partition['Validation'][1], **params)

#print(len(training_generator.get_indexes()))
#training_generator.on_epoch_end()

#print(training_generator.__len__())
